# Base de dados usada para a realização desse projeto

[COVID-19 Cases, hospitalizations and deaths, County](https://catalog.data.gov/dataset/covid-19-cases-hospitalizations-and-deaths-by-county/resource/9151346d-21bd-4f13-a7e2-ac27103ef698)

In [ ]:
# Instalando Biblioteca Pandera

!pip install pandera


# Importando Bibliotecas para a utlização do pandas, pandera, ler arquivos json e acessar o bucket do GCP

In [ ]:
# Importando Bibliotecas

import pandas as pd
import pandera as pa
import json
from google.cloud import storage


# Utilizando chave de Serviço do GCP

In [ ]:
storage_client = storage.Client.from_service_account_json('/content/drive/MyDrive/Dados/Projeto_Final/chave_projeto-finalsc01-b94741ddd471.json')


# Acessando Bucket do Projeto

In [ ]:
bucket_name = 'projeto-finalsc01'
bucket = storage_client.get_bucket(bucket_name)


# Listando Arquivos no Bucket

In [ ]:
filename = list(bucket.list_blobs(prefix=''))
for name in filename:
  print(name.name)


# Carregando Dataset json

In [ ]:
blob = bucket.blob('dados-brutos/COVID_US_HOSPITALIZATION.json')
data = json.loads(blob.download_as_string(client=None))


# Criando DataFrame ordenado por Dados utilizando pandas

In [ ]:
df = pd.DataFrame(data["data"])
df


# Analisando a base de dados

In [ ]:
print(df.info())


# Vizualização do schema

In [ ]:
pd.DataFrame(data['meta']['view']["columns"])


# Deletando colunas que não fazem sentido para a análise de dados

In [ ]:
df.drop([0,1,2,3,4,5,6,7,], axis = 1, inplace=True)


# Traduzindo o Schema para Português-BR

In [ ]:
df.rename(columns={
    8:'Data_Atualizacao',
    9:'Condado_Codigo',
    10:'Condado',
    11:'Casos_Total',
    12:'Casos_Confirmados',
    13:'Casos_Provaveis',
    14:'Casos_Total_Taxa',
    15:'Casos_Hospitalizados',
    16:'Obitos_Total',
    17:'Obitos_Confirmados',
    18:'Obitos_Provaveis'

}, inplace=True)


# Verificando os valores nulos

In [ ]:
df.isna().sum()


# Substituindo valores nulos por 0.

In [ ]:
df.replace([None], '0', inplace=True)


# Modificando o tipo de dado de algumas colunas para fazer a validação

In [ ]:
# Alterando tipo de dados da Coluna 'Data_Atualizacao' para datetime

df['Data_Atualizacao'] =  pd.to_datetime(df['Data_Atualizacao'])


In [ ]:
# Alterando tipo de dados da Coluna 'Condado' para String

df[['Condado']] = df[['Condado']].astype(str)


In [ ]:
# Listando Dados Únicos na Coluna 'Condado'

sorted(pd.unique(df['Condado']))


In [ ]:
# Procurar um dado em uma coluna e alterar
# Acrescentar 'County aos nomes dos Condados


df.loc[df.Condado == 'Fairfield', ['Condado']] = 'Fairfield County'
df.loc[df.Condado == 'Hartford', ['Condado']] = 'Hartford County'
df.loc[df.Condado == 'Litchfield', ['Condado']] = 'Litchfield County'
df.loc[df.Condado == 'Middlesex', ['Condado']] = 'Middlesex County'
df.loc[df.Condado == 'New Haven', ['Condado']] = 'New Haven County'
df.loc[df.Condado == 'New London', ['Condado']] = 'New London County'
df.loc[df.Condado == 'Tolland', ['Condado']] = 'Tolland County'
df.loc[df.Condado == 'Windham', ['Condado']] = 'Windham County'


In [ ]:
# Alterando tipo de dados da Coluna 'Condado_Codigo' para Int

df[['Condado_Codigo']] = df[['Condado_Codigo']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Casos_Total' para Int

df[['Casos_Total']] = df[['Casos_Total']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Casos_Confirmados' para Int

df[['Casos_Confirmados']] = df[['Casos_Confirmados']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Casos_Provaveis' para Int

df[['Casos_Provaveis']] = df[['Casos_Provaveis']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Casos_Total_Taxa' para Float

df[['Casos_Total_Taxa']] = df[['Casos_Total_Taxa']].astype(float)


In [ ]:
# Alterando tipo de dados da Coluna 'Casos_Hospitalizados' para Int

df[['Casos_Hospitalizados']] = df[['Casos_Hospitalizados']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Obitos_Total' para Int

df[['Obitos_Total']] = df[['Obitos_Total']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Obitos_Confirmados' para Int

df[['Obitos_Confirmados']] = df[['Obitos_Confirmados']].astype(int)


In [ ]:
# Alterando tipo de dados da Coluna 'Obitos_Provaveis' para Int

df[['Obitos_Provaveis']] = df[['Obitos_Provaveis']].astype(int)


# Validando os dados do DataFrame usando a biblioteca panderas.

In [ ]:
# Validando Dados do Dataframe

schema = pa.DataFrameSchema(
    columns = {
        "Data_Atualizacao":pa.Column(pa.DateTime),
        "Condado_Codigo":pa.Column(pa.Int),
        "Condado":pa.Column(pa.String),
        "Casos_Total":pa.Column(pa.Int),
        "Casos_Confirmados":pa.Column(pa.Int),
        "Casos_Provaveis":pa.Column(pa.Int),
        "Casos_Total_Taxa":pa.Column(pa.Float),
        "Casos_Hospitalizados":pa.Column(pa.Int),
        "Obitos_Total":pa.Column(pa.Int),
        "Obitos_Confirmados":pa.Column(pa.Int),
        "Obitos_Provaveis":pa.Column(pa.Int)
    }
)

schema.validate(df)


# Conferindo Dados do Dataframe

In [ ]:
# Listando tipos de dados nas Colunas

print(df.info())


# Criando DataFrames Únicos para cada Condado e salvando no Bucket do GCP

In [ ]:
# Criando DataFrame para o Condado Fairfield County

df_Fairfield_County = df.loc[df.Condado == 'Fairfield County']
df_Fairfield_County


In [ ]:
# Salvando DataFrame do Condado Fairfield County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Fairfield_County_Hospitalizacoes.csv').upload_from_string(df_Fairfield_County.to_csv(index=False),'Fairfield_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado Hartford County

df_Hartford_County = df.loc[df.Condado == 'Hartford County']
df_Hartford_County


In [ ]:
# Salvando DataFrame do Condado Hartford County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Hartford_County_Hospitalizacoes.csv').upload_from_string(df_Hartford_County.to_csv(index=False),'Hartford_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado Litchfield County

df_Litchfield_County = df.loc[df.Condado == 'Litchfield County']
df_Litchfield_County


In [ ]:
# Salvando DataFrame do Condado Litchfield County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Litchfield_County_Hospitalizacoes.csv').upload_from_string(df_Litchfield_County.to_csv(index=False),'Litchfield_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado Middlesex County

df_Middlesex_County = df.loc[df.Condado == 'Middlesex County']
df_Middlesex_County


In [ ]:
# Salvando DataFrame do Condado Middlesex County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Middlesex_County_Hospitalizacoes.csv').upload_from_string(df_Middlesex_County.to_csv(index=False),'Middlesex_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado New Haven County

df_New_Haven_County = df.loc[df.Condado == 'New Haven County']
df_New_Haven_County


In [ ]:
# Salvando DataFrame do Condado New Haven County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/New_Haven_County_Hospitalizacoes.csv').upload_from_string(df_New_Haven_County.to_csv(index=False),'New_Haven_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado New London County

df_New_London_County = df.loc[df.Condado == 'New London County']
df_New_London_County


In [ ]:
# Salvando DataFrame do Condado New London County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/New_London_County_Hospitalizacoes.csv').upload_from_string(df_New_London_County.to_csv(index=False),'New_London_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado Tolland County

df_Tolland_County = df.loc[df.Condado == 'Tolland County']
df_Tolland_County


In [ ]:
# Salvando DataFrame do Condado Tolland County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Tolland_County_Hospitalizacoes.csv').upload_from_string(df_Tolland_County.to_csv(index=False),'Tolland_County_Hospitalizacoes.csv' )

In [ ]:
# Criando DataFrame para o Condado Windham County

df_Windham_County = df.loc[df.Condado == 'Windham County']
df_Windham_County


In [ ]:
# Salvando DataFrame do Condado Windham County no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Windham_County_Hospitalizacoes.csv').upload_from_string(df_Windham_County.to_csv(index=False),'Windham_County_Hospitalizacoes.csv' )

# Salvando as transformações realizadas no DataFrame para o Bucket do GCP

In [ ]:
# Salvando DataFrame normalizado no Bucket do GCP

bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Casos_Hospitalizacoes_Obitos_Cidade.csv').upload_from_string(df.to_csv(index=False),'Casos_Hospitalizacoes_Obitos_Cidade.csv' )